### Using the Census API to get zip-code level population, median income, pct_below_poverty and racial data for NYC zip codes.

In [3]:
import json
from census import Census
from us import states
from us import states
import glob
from pprint import pprint
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import pandas as pd


In [4]:
# load keys
with open('keys.json') as f:
    keys = json.load(f)

# create census object
c = Census(keys['census_api_key'])

In [3]:
# define a function to get data from a particular table
def get_from_census(table_id, year):
    response = requests.get(f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,group({table_id})&for=zip%20code%20tabulation%20area:*&key={keys['census_api_key']}")
    data = response.json()
    df = pd.DataFrame(data, columns=data[0]).drop(0)
    return df

# Race

In [4]:
# get race data for 2021
race_2021 = get_from_census("B02001", "2021")
race_2021


,NAME,B02001_001E,B02001_001EA,B02001_001M,B02001_001MA,B02001_002E,B02001_002EA,B02001_002M,B02001_002MA,B02001_003E,...,B02001_009EA,B02001_009M,B02001_009MA,B02001_010E,B02001_010EA,B02001_010M,B02001_010MA,GEO_ID,NAME,zip code tabulation area
1,ZCTA5 00601,17126,None,429,None,14463,None,626,None,243,...,None,323,None,111,None,127,None,860Z200US00601,ZCTA5 00601,00601
2,ZCTA5 00602,37895,None,279,None,23237,None,1340,None,688,...,None,629,None,10046,None,1186,None,860Z200US00602,ZCTA5 00602,00602
3,ZCTA5 00603,49136,None,841,None,36497,None,1337,None,1681,...,None,764,None,772,None,274,None,860Z200US00603,ZCTA5 00603,00603
4,ZCTA5 00606,5751,None,355,None,3319,None,479,None,27,...,None,160,None,18,None,30,None,860Z200US00606,ZCTA5 00606,00606
5,ZCTA5 00610,26153,None,382,None,15815,None,1081,None,642,...,None,573,None,6254,None,1022,None,860Z200US00610,ZCTA5 00610,00610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33770,ZCTA5 99923,13,None,20,None,13,None,20,None,0,...,None,10,None,0,None,10,None,860Z200US99923,ZCTA5 99923,99923
33771,ZCTA5 99925,917,None,118,None,411,None,78,None,4,...,None,10,None,153,None,41,None,860Z200US99925,ZCTA5 99925,99925
33772,ZCTA5 99926,1445,None,155,None,92,None,37,None,0,...,None,12,None,22,None,24,None,860Z200US99926,ZCTA5 99926,99926
33773,ZCTA5 99927,11,None,15,None,11,None,15,None,0,...,None,10,None,0,None,10,None,860Z200US99927,ZCTA5 99927,99927


In [5]:
# B02001_001E - Total
# B02001_002E - White alone
# B02001_003E - Black or African American alone
# B02001_004E - American Indian and Alaska Native alone
# B02001_005E - Asian alone
# B02001_006E - Native Hawaiian and Other Pacific Islander alone

In [6]:
# get racial data for 2021
racial_2021 = get_from_census("B02001", "2021")

# clean it up
racial_2021 = racial_2021[['zip code tabulation area', 'B02001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 'B02001_006E']]
racial_2021.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E
1,00601,17126,14463,243,13,2,0
2,00602,37895,23237,688,44,46,0
3,00603,49136,36497,1681,54,38,0
4,00606,5751,3319,27,21,0,0
5,00610,26153,15815,642,0,0,0
...,...,...,...,...,...,...,...
33770,99923,13,13,0,0,0,0
33771,99925,917,411,4,335,7,0
33772,99926,1445,92,0,1304,6,0
33773,99927,11,11,0,0,0,0


In [7]:
racial_2021 = racial_2021.rename(columns={
    'zip code tabulation area': 'zip',
    'B02001_001E': 'total',
    'B02001_002E': 'white',
    'B02001_003E': 'black_or_african_american',
    'B02001_004E': 'american_indian_or_alaska_native',
    'B02001_005E': 'asian',
    'B02001_006E': 'native_hawaiian_and_other_pacific_islander'
})

In [8]:
# convert all columns to numeric
racial_2021 = racial_2021.apply(pd.to_numeric)


In [9]:
racial_2021.dtypes

zip                                           int64
total                                         int64
white                                         int64
black_or_african_american                     int64
american_indian_or_alaska_native              int64
asian                                         int64
native_hawaiian_and_other_pacific_islander    int64
dtype: object

In [10]:
# calculate percentages
racial_2021['pct_white'] = racial_2021['white'] / racial_2021['total'] * 100
racial_2021['pct_black_or_african_american'] = racial_2021['black_or_african_american'] / racial_2021['total'] * 100
racial_2021['pct_american_indian_or_alaska_native'] = racial_2021['american_indian_or_alaska_native'] / racial_2021['total'] * 100
racial_2021['pct_asian'] = racial_2021['asian'] / racial_2021['total'] * 100
racial_2021['pct_native_hawaiian_and_other_pacific_islander'] = racial_2021['native_hawaiian_and_other_pacific_islander'] / racial_2021['total'] * 100
racial_2021


,zip,total,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander
1,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000
2,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000
3,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000
4,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000
5,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
33770,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000
33771,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000
33772,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000
33773,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
racial_2021.dtypes



zip                                                 int64
total                                               int64
white                                               int64
black_or_african_american                           int64
american_indian_or_alaska_native                    int64
asian                                               int64
native_hawaiian_and_other_pacific_islander          int64
pct_white                                         float64
pct_black_or_african_american                     float64
pct_american_indian_or_alaska_native              float64
pct_asian                                         float64
pct_native_hawaiian_and_other_pacific_islander    float64
dtype: object

In [12]:
# get hispanic data for 2021
hispanic_2021 = get_from_census("B03002", "2021")
hispanic_2021


,NAME,B03002_001E,B03002_001EA,B03002_001M,B03002_001MA,B03002_002E,B03002_002EA,B03002_002M,B03002_002MA,B03002_003E,...,B03002_020EA,B03002_020M,B03002_020MA,B03002_021E,B03002_021EA,B03002_021M,B03002_021MA,GEO_ID,NAME,zip code tabulation area
1,ZCTA5 00601,17126,None,429,None,88,None,71,None,88,...,None,323,None,111,None,127,None,860Z200US00601,ZCTA5 00601,00601
2,ZCTA5 00602,37895,None,279,None,2246,None,442,None,1770,...,None,629,None,9714,None,1168,None,860Z200US00602,ZCTA5 00602,00602
3,ZCTA5 00603,49136,None,841,None,1015,None,395,None,731,...,None,765,None,678,None,246,None,860Z200US00603,ZCTA5 00603,00603
4,ZCTA5 00606,5751,None,355,None,41,None,52,None,41,...,None,160,None,18,None,30,None,860Z200US00606,ZCTA5 00606,00606
5,ZCTA5 00610,26153,None,382,None,1100,None,331,None,920,...,None,573,None,6233,None,1020,None,860Z200US00610,ZCTA5 00610,00610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33770,ZCTA5 99923,13,None,20,None,13,None,20,None,13,...,None,10,None,0,None,10,None,860Z200US99923,ZCTA5 99923,99923
33771,ZCTA5 99925,917,None,118,None,879,None,116,None,392,...,None,10,None,10,None,12,None,860Z200US99925,ZCTA5 99925,99925
33772,ZCTA5 99926,1445,None,155,None,1398,None,146,None,89,...,None,12,None,0,None,10,None,860Z200US99926,ZCTA5 99926,99926
33773,ZCTA5 99927,11,None,15,None,11,None,15,None,11,...,None,10,None,0,None,10,None,860Z200US99927,ZCTA5 99927,99927


In [13]:
# B03002_001E - Total
# B03002_002E - Not Hispanic or Latino


In [14]:
hispanic_2021 = hispanic_2021[['zip code tabulation area', 'B03002_001E', 'B03002_002E']]
hispanic_2021 = hispanic_2021.rename(columns={
    'zip code tabulation area': 'zip',
})
hispanic_2021

,zip,B03002_001E,B03002_002E
1,00601,17126,88
2,00602,37895,2246
3,00603,49136,1015
4,00606,5751,41
5,00610,26153,1100
...,...,...,...
33770,99923,13,13
33771,99925,917,879
33772,99926,1445,1398
33773,99927,11,11


In [19]:
hispanic_2021 = hispanic_2021.rename(columns={
    'B03002_001E': 'total',
    'B03002_002E': 'not_hispanic_or_latino',
})

hispanic_2021

,zip,total,not_hispanic_or_latino
1,601,17126,88
2,602,37895,2246
3,603,49136,1015
4,606,5751,41
5,610,26153,1100
...,...,...,...
33770,99923,13,13
33771,99925,917,879
33772,99926,1445,1398
33773,99927,11,11


In [20]:
# convert all columns to numeric
hispanic_2021 = hispanic_2021.apply(pd.to_numeric)


In [21]:
hispanic_2021['hispanic_or_latino'] = hispanic_2021['total'] - hispanic_2021['not_hispanic_or_latino']
hispanic_2021['pct_hispanic_or_latino'] = hispanic_2021['hispanic_or_latino'] / hispanic_2021['total'] * 100
hispanic_2021

,zip,total,not_hispanic_or_latino,hispanic_or_latino,pct_hispanic_or_latino
1,601,17126,88,17038,99.486161
2,602,37895,2246,35649,94.073097
3,603,49136,1015,48121,97.934305
4,606,5751,41,5710,99.287081
5,610,26153,1100,25053,95.793982
...,...,...,...,...,...
33770,99923,13,13,0,0.000000
33771,99925,917,879,38,4.143948
33772,99926,1445,1398,47,3.252595
33773,99927,11,11,0,0.000000


In [22]:
# merge the two dataframes
racial_data_2021 = pd.merge(racial_2021, hispanic_2021, on='zip')
racial_data_2021


,zip,total_x,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander,total_y,not_hispanic_or_latino,hispanic_or_latino,pct_hispanic_or_latino
0,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000,17126,88,17038,99.486161
1,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000,37895,2246,35649,94.073097
2,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000,49136,1015,48121,97.934305
3,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000,5751,41,5710,99.287081
4,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000,26153,1100,25053,95.793982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33769,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,13,13,0,0.000000
33770,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000,917,879,38,4.143948
33771,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000,1445,1398,47,3.252595
33772,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,11,11,0,0.000000


In [23]:
# remove columns
racial_data_2021 = racial_data_2021.drop(columns=['total_y', 'not_hispanic_or_latino'])
racial_data_2021 = racial_data_2021.rename(columns={'total_x': 'total_race'})
racial_data_2021

,zip,total_race,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander,hispanic_or_latino,pct_hispanic_or_latino
0,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000,17038,99.486161
1,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000,35649,94.073097
2,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000,48121,97.934305
3,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000,5710,99.287081
4,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000,25053,95.793982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33769,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
33770,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000,38,4.143948
33771,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000,47,3.252595
33772,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000


In [24]:
# rename columns 
racial_data_2021 = racial_data_2021.rename(columns={
    'total_race': 'total_race_2021',
    'white': 'white_2021',
    'black_or_african_american': 'black_or_african_american_2021',
    'american_indian_or_alaska_native': 'american_indian_or_alaska_native_2021',
    'asian': 'asian_2021',
    'native_hawaiian_and_other_pacific_islander': 'native_hawaiian_and_other_pacific_islander_2021',
    'pct_white': 'pct_white_2021',
    'pct_black_or_african_american': 'pct_black_or_african_american_2021',
    'pct_american_indian_or_alaska_native': 'pct_american_indian_or_alaska_native_2021',
    'pct_asian': 'pct_asian_2021',
    'pct_native_hawaiian_and_other_pacific_islander': 'pct_native_hawaiian_and_other_pacific_islander_2021',
    'hispanic_or_latino': 'hispanic_or_latino_2021',
    'pct_hispanic_or_latino': 'pct_hispanic_or_latino_2021'
})
racial_data_2021


,zip,total_race_2021,white_2021,black_or_african_american_2021,american_indian_or_alaska_native_2021,asian_2021,native_hawaiian_and_other_pacific_islander_2021,pct_white_2021,pct_black_or_african_american_2021,pct_american_indian_or_alaska_native_2021,pct_asian_2021,pct_native_hawaiian_and_other_pacific_islander_2021,hispanic_or_latino_2021,pct_hispanic_or_latino_2021
0,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000,17038,99.486161
1,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000,35649,94.073097
2,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000,48121,97.934305
3,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000,5710,99.287081
4,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000,25053,95.793982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33769,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
33770,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000,38,4.143948
33771,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000,47,3.252595
33772,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000


In [25]:
# get race data for 2020
racial_2020 = get_from_census("B02001", "2020")
racial_2020 = racial_2020[['zip code tabulation area', 'B02001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 'B02001_006E']]
racial_2020 = racial_2020.rename(columns={
    'zip code tabulation area': 'zip',
    'B02001_001E': 'total',
    'B02001_002E': 'white',
    'B02001_003E': 'black_or_african_american',
    'B02001_004E': 'american_indian_or_alaska_native',
    'B02001_005E': 'asian',
    'B02001_006E': 'native_hawaiian_and_other_pacific_islander'
})
racial_2020 = racial_2020.apply(pd.to_numeric)
racial_2020





,zip,total,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander
1,29590,3543,718,2617,27,0,0
2,93306,74296,51869,1727,1208,2455,178
3,93660,4082,2231,40,0,0,0
4,93110,15777,12042,311,287,674,0
5,93212,22596,13033,2708,333,177,23
...,...,...,...,...,...,...,...
33116,16623,552,550,0,0,0,0
33117,16627,2118,1980,9,2,0,0
33118,16634,315,311,0,0,4,0
33119,16640,707,701,3,0,3,0


In [26]:
racial_2020['pct_white'] = racial_2020['white'] / racial_2020['total'] * 100
racial_2020['pct_black_or_african_american'] = racial_2020['black_or_african_american'] / racial_2020['total'] * 100
racial_2020['pct_american_indian_or_alaska_native'] = racial_2020['american_indian_or_alaska_native'] / racial_2020['total'] * 100
racial_2020['pct_asian'] = racial_2020['asian'] / racial_2020['total'] * 100
racial_2020['pct_native_hawaiian_and_other_pacific_islander'] = racial_2020['native_hawaiian_and_other_pacific_islander'] / racial_2020['total'] * 100
racial_2020

,zip,total,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander
1,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,0.000000,0.000000
2,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,3.304350,0.239582
3,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,0.000000,0.000000
4,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,4.272042,0.000000
5,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,0.783324,0.101788
...,...,...,...,...,...,...,...,...,...,...,...,...
33116,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,0.000000,0.000000
33117,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,0.000000,0.000000
33118,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,1.269841,0.000000
33119,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,0.424328,0.000000


In [27]:
# get hispanic data for 2020
hispanic_2020 = get_from_census("B03002", "2020")
hispanic_2020 = hispanic_2020[['zip code tabulation area', 'B03002_001E', 'B03002_002E']]
hispanic_2020 = hispanic_2020.rename(columns={
    'zip code tabulation area': 'zip',
})
hispanic_2020



,zip,B03002_001E,B03002_002E
1,29590,3543,3197
2,93306,74296,26180
3,93660,4082,157
4,93110,15777,11526
5,93212,22596,6955
...,...,...,...
33116,16623,552,552
33117,16627,2118,2114
33118,16634,315,315
33119,16640,707,702


In [28]:
hispanic_2020 = hispanic_2020.rename(columns={
    'B03002_001E': 'total',
    'B03002_002E': 'not_hispanic_or_latino',
})

hispanic_2020

,zip,total,not_hispanic_or_latino
1,29590,3543,3197
2,93306,74296,26180
3,93660,4082,157
4,93110,15777,11526
5,93212,22596,6955
...,...,...,...
33116,16623,552,552
33117,16627,2118,2114
33118,16634,315,315
33119,16640,707,702


In [29]:
hispanic_2020 = hispanic_2020.apply(pd.to_numeric)
hispanic_2020['hispanic_or_latino'] = hispanic_2020['total'] - hispanic_2020['not_hispanic_or_latino']
hispanic_2020['pct_hispanic_or_latino'] = hispanic_2020['hispanic_or_latino'] / hispanic_2020['total'] * 100
hispanic_2020

,zip,total,not_hispanic_or_latino,hispanic_or_latino,pct_hispanic_or_latino
1,29590,3543,3197,346,9.765735
2,93306,74296,26180,48116,64.762571
3,93660,4082,157,3925,96.153846
4,93110,15777,11526,4251,26.944286
5,93212,22596,6955,15641,69.220216
...,...,...,...,...,...
33116,16623,552,552,0,0.000000
33117,16627,2118,2114,4,0.188857
33118,16634,315,315,0,0.000000
33119,16640,707,702,5,0.707214


In [30]:
# merge the two dataframes
racial_data_2020 = pd.merge(racial_2020, hispanic_2020, on='zip')
racial_data_2020 = racial_data_2020.drop(columns=['total_y', 'not_hispanic_or_latino'])
racial_data_2020 = racial_data_2020.rename(columns={'total_x': 'total_race'})
racial_data_2020

,zip,total_race,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander,hispanic_or_latino,pct_hispanic_or_latino
0,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,0.000000,0.000000,346,9.765735
1,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,3.304350,0.239582,48116,64.762571
2,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,0.000000,0.000000,3925,96.153846
3,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,4.272042,0.000000,4251,26.944286
4,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,0.783324,0.101788,15641,69.220216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,0.000000,0.000000,0,0.000000
33116,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,0.000000,0.000000,4,0.188857
33117,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,1.269841,0.000000,0,0.000000
33118,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,0.424328,0.000000,5,0.707214


In [31]:
# rename columns 
racial_data_2020 = racial_data_2020.rename(columns={
    'total_race': 'total_race_2020',
    'white': 'white_2020',
    'black_or_african_american': 'black_or_african_american_2020',
    'american_indian_or_alaska_native': 'american_indian_or_alaska_native_2020',
    'asian': 'asian_2020',
    'native_hawaiian_and_other_pacific_islander': 'native_hawaiian_and_other_pacific_islander_2020',
    'pct_white': 'pct_white_2020',
    'pct_black_or_african_american': 'pct_black_or_african_american_2020',
    'pct_american_indian_or_alaska_native': 'pct_american_indian_or_alaska_native_2020',
    'pct_asian': 'pct_asian_2020',
    'pct_native_hawaiian_and_other_pacific_islander': 'pct_native_hawaiian_and_other_pacific_islander_2020',
    'hispanic_or_latino': 'hispanic_or_latino_2020',
    'pct_hispanic_or_latino': 'pct_hispanic_or_latino_2020'
})
racial_data_2020

,zip,total_race_2020,white_2020,black_or_african_american_2020,american_indian_or_alaska_native_2020,asian_2020,native_hawaiian_and_other_pacific_islander_2020,pct_white_2020,pct_black_or_african_american_2020,pct_american_indian_or_alaska_native_2020,pct_asian_2020,pct_native_hawaiian_and_other_pacific_islander_2020,hispanic_or_latino_2020,pct_hispanic_or_latino_2020
0,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,0.000000,0.000000,346,9.765735
1,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,3.304350,0.239582,48116,64.762571
2,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,0.000000,0.000000,3925,96.153846
3,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,4.272042,0.000000,4251,26.944286
4,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,0.783324,0.101788,15641,69.220216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,0.000000,0.000000,0,0.000000
33116,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,0.000000,0.000000,4,0.188857
33117,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,1.269841,0.000000,0,0.000000
33118,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,0.424328,0.000000,5,0.707214


In [32]:
# merge racial data for 2020 and 2021

racial_data = pd.merge(racial_data_2020, racial_data_2021, on='zip')
racial_data

,zip,total_race_2020,white_2020,black_or_african_american_2020,american_indian_or_alaska_native_2020,asian_2020,native_hawaiian_and_other_pacific_islander_2020,pct_white_2020,pct_black_or_african_american_2020,pct_american_indian_or_alaska_native_2020,...,american_indian_or_alaska_native_2021,asian_2021,native_hawaiian_and_other_pacific_islander_2021,pct_white_2021,pct_black_or_african_american_2021,pct_american_indian_or_alaska_native_2021,pct_asian_2021,pct_native_hawaiian_and_other_pacific_islander_2021,hispanic_or_latino_2021,pct_hispanic_or_latino_2021
0,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,...,25,14,0,23.286517,72.584270,0.702247,0.393258,0.000000,299,8.398876
1,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,...,1204,2447,117,61.578411,3.320003,1.615717,3.283770,0.157009,49268,66.115569
2,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,...,28,0,0,44.589650,0.967067,0.731835,0.000000,0.000000,3676,96.079456
3,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,...,210,863,6,73.049862,1.211380,1.284797,5.279902,0.036708,4481,27.415112
4,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,...,500,237,42,47.464144,13.772248,2.160014,1.023847,0.181441,16159,69.807327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32918,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,...,0,0,0,99.823944,0.000000,0.000000,0.000000,0.000000,0,0.000000
32919,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,...,1,0,1,88.642413,0.532387,0.044366,0.000000,0.044366,13,0.576752
32920,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,...,0,11,0,93.818182,0.000000,0.000000,4.000000,0.000000,0,0.000000
32921,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,...,0,2,0,98.757062,0.225989,0.000000,0.225989,0.000000,2,0.225989


In [34]:
# save to csv
racial_data.to_csv('racial_data.csv', index=False)

# Percenatge below poverty

In [ ]:
# get poverty data for 2021
poverty_2021 = get_from_census("B17017", "2021")

# clean it up
poverty_2021 = poverty_2021[['zip code tabulation area', 'B17017_002E', 'B17017_001E']]
poverty_2021.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B17017_002E,B17017_001E
1,00601,3520,5397
2,00602,6300,12858
3,00603,10047,19295
4,00606,957,1968
5,00610,4222,8934
...,...,...,...
33770,99923,0,13
33771,99925,63,345
33772,99926,55,457
33773,99927,0,11


In [ ]:
# rename columns
poverty_2021.rename(columns={'B17017_002E': 'households_below_poverty', 'B17017_001E': 'total_households', 'zip code tabulation area': 'zip'}, inplace=True)



In [ ]:
poverty_2021

,zip,households_below_poverty,total_households
1,00601,3520,5397
2,00602,6300,12858
3,00603,10047,19295
4,00606,957,1968
5,00610,4222,8934
...,...,...,...
33770,99923,0,13
33771,99925,63,345
33772,99926,55,457
33773,99927,0,11


In [ ]:
poverty_2021['pct_below_poverty_2021'] = poverty_2021['households_below_poverty'].astype(int) / poverty_2021['total_households'].astype(int) * 100
poverty_2021

,zip,households_below_poverty,total_households,pct_below_poverty_2021
1,00601,3520,5397,65.221419
2,00602,6300,12858,48.996734
3,00603,10047,19295,52.070485
4,00606,957,1968,48.628049
5,00610,4222,8934,47.257667
...,...,...,...,...
33770,99923,0,13,0.000000
33771,99925,63,345,18.260870
33772,99926,55,457,12.035011
33773,99927,0,11,0.000000


In [ ]:
# get poverty data for 2020 
poverty_2020 = get_from_census("B17017", "2020")

# clean it up
poverty_2020 = poverty_2020[['zip code tabulation area', 'B17017_002E', 'B17017_001E']]
poverty_2020.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B17017_002E,B17017_001E
1,29590,333,986
2,93306,4006,21974
3,93660,288,949
4,93110,749,6385
5,93212,1074,4014
...,...,...,...
33116,16623,29,216
33117,16627,138,880
33118,16634,14,146
33119,16640,49,299


In [ ]:
poverty_2020.rename(columns={'B17017_002E': 'households_below_poverty', 'B17017_001E': 'total_households', 'zip code tabulation area': 'zip'}, inplace=True)


In [ ]:
poverty_2020['pct_below_poverty_2020'] = poverty_2020['households_below_poverty'].astype(int) / poverty_2020['total_households'].astype(int) * 100
poverty_2020

,zip,households_below_poverty,total_households,pct_below_poverty_2020
1,29590,333,986,33.772819
2,93306,4006,21974,18.230636
3,93660,288,949,30.347734
4,93110,749,6385,11.730619
5,93212,1074,4014,26.756353
...,...,...,...,...
33116,16623,29,216,13.425926
33117,16627,138,880,15.681818
33118,16634,14,146,9.589041
33119,16640,49,299,16.387960


In [ ]:
pct_below_poverty = pd.merge(poverty_2020, poverty_2021, on='zip', how='outer')
pct_below_poverty

,zip,households_below_poverty_x,total_households_x,pct_below_poverty_2020,households_below_poverty_y,total_households_y,pct_below_poverty_2021
0,29590,333,986,33.772819,309,908,34.030837
1,93306,4006,21974,18.230636,3918,21818,17.957650
2,93660,288,949,30.347734,291,868,33.525346
3,93110,749,6385,11.730619,670,6474,10.349089
4,93212,1074,4014,26.756353,1112,4230,26.288416
...,...,...,...,...,...,...,...
33966,99635,NaN,NaN,NaN,0,0,NaN
33967,99675,NaN,NaN,NaN,6,11,54.545455
33968,99707,NaN,NaN,NaN,0,0,NaN
33969,99725,NaN,NaN,NaN,57,314,18.152866


In [ ]:
# drop columns

pct_below_poverty.drop(columns=['households_below_poverty_x', 'total_households_x', 'households_below_poverty_y', 'total_households_y'], inplace=True)



In [ ]:
# rename columns

pct_below_poverty.rename(columns={'pct_below_poverty_2020_x': 'pct_below_poverty_2020', 'pct_below_poverty_2020_y': 'pct_below_poverty_2021'}, inplace=True)
pct_below_poverty

,zip,pct_below_poverty_2020,pct_below_poverty_2021
0,29590,33.772819,34.030837
1,93306,18.230636,17.957650
2,93660,30.347734,33.525346
3,93110,11.730619,10.349089
4,93212,26.756353,26.288416
...,...,...,...
33966,99635,NaN,NaN
33967,99675,NaN,54.545455
33968,99707,NaN,NaN
33969,99725,NaN,18.152866


In [ ]:
# save to csv

# pct_below_poverty.to_csv('pct_below_poverty.csv', index=False)

# Median income 

In [ ]:
# get income data for 2021
median_income_2021 = get_from_census("B19013", "2021")

# clean it up
median_income_2021 = median_income_2021[['zip code tabulation area', 'B19013_001E']]
median_income_2021.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B19013_001E
1,00601,15292
2,00602,18716
3,00603,16789
4,00606,18835
5,00610,21239
...,...,...
33770,99923,-666666666
33771,99925,70625
33772,99926,58229
33773,99927,-666666666


In [ ]:
# create a column for the year
median_income_2021['year'] = 2021
median_income_2021

,zip code tabulation area,B19013_001E,year
1,00601,15292,2021
2,00602,18716,2021
3,00603,16789,2021
4,00606,18835,2021
5,00610,21239,2021
...,...,...,...
33770,99923,-666666666,2021
33771,99925,70625,2021
33772,99926,58229,2021
33773,99927,-666666666,2021


In [ ]:
# rename zip code tabulation area to zip    
median_income_2021.rename(columns={'zip code tabulation area': 'zip', 'B19013_001E': 'med_inc_2021'}, inplace=True)
median_income_2021

,zip,med_inc_2021,year
1,00601,15292,2021
2,00602,18716,2021
3,00603,16789,2021
4,00606,18835,2021
5,00610,21239,2021
...,...,...,...
33770,99923,-666666666,2021
33771,99925,70625,2021
33772,99926,58229,2021
33773,99927,-666666666,2021


In [ ]:
# get income data for 2020
median_income_2020 = get_from_census("B19013", "2020")

# clean it up
median_income_2020 = median_income_2020[['zip code tabulation area', 'B19013_001E']]
median_income_2020.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B19013_001E
1,29590,30985
2,93306,54450
3,93660,39625
4,93110,93264
5,93212,42983
...,...,...
33116,16623,51667
33117,16627,45000
33118,16634,51500
33119,16640,55982


In [ ]:
# create a column for the year
median_income_2020['year'] = 2020
median_income_2020

,zip code tabulation area,B19013_001E,year
1,29590,30985,2020
2,93306,54450,2020
3,93660,39625,2020
4,93110,93264,2020
5,93212,42983,2020
...,...,...,...
33116,16623,51667,2020
33117,16627,45000,2020
33118,16634,51500,2020
33119,16640,55982,2020


In [ ]:
# rename zip code tabulation area to zip    
median_income_2020.rename(columns={'zip code tabulation area': 'zip', 'B19013_001E': 'med_inc_2020'}, inplace=True)
median_income_2020

,zip,med_inc_2020,year
1,29590,30985,2020
2,93306,54450,2020
3,93660,39625,2020
4,93110,93264,2020
5,93212,42983,2020
...,...,...,...
33116,16623,51667,2020
33117,16627,45000,2020
33118,16634,51500,2020
33119,16640,55982,2020


In [ ]:
median_income = pd.merge(median_income_2020, median_income_2021, on='zip', how='outer')
median_income

,zip,med_inc_2020,year_x,med_inc_2021,year_y
0,29590,30985,2020.0,52679,2021.0
1,93306,54450,2020.0,60857,2021.0
2,93660,39625,2020.0,40000,2021.0
3,93110,93264,2020.0,99261,2021.0
4,93212,42983,2020.0,46312,2021.0
...,...,...,...,...,...
33966,99635,NaN,NaN,-666666666,2021.0
33967,99675,NaN,NaN,34375,2021.0
33968,99707,NaN,NaN,-666666666,2021.0
33969,99725,NaN,NaN,45931,2021.0


In [ ]:
# drop columns year_x and year_y

median_income.drop(columns=['year_x', 'year_y'], inplace=True)
median_income

,zip,med_inc_2020,med_inc_2021
0,29590,30985,52679
1,93306,54450,60857
2,93660,39625,40000
3,93110,93264,99261
4,93212,42983,46312
...,...,...,...
33966,99635,NaN,-666666666
33967,99675,NaN,34375
33968,99707,NaN,-666666666
33969,99725,NaN,45931


# Population

In [ ]:
# get population data for 2021
population_2021 = get_from_census("B01003", "2021")

# clean it up
population_2021 = population_2021\
        [['zip code tabulation area', 'B01003_001E']]\
        .rename(columns={'zip code tabulation area': 'zip'})

population_2021

,zip,B01003_001E
1,00601,17126
2,00602,37895
3,00603,49136
4,00606,5751
5,00610,26153
...,...,...
33770,99923,13
33771,99925,917
33772,99926,1445
33773,99927,11


In [ ]:
#rename columns
population_2021.rename(columns={'B01003_001E': 'pop_2021'}, inplace=True)
population_2021

,zip,pop_2021
1,00601,17126
2,00602,37895
3,00603,49136
4,00606,5751
5,00610,26153
...,...,...
33770,99923,13
33771,99925,917
33772,99926,1445
33773,99927,11


In [ ]:
# get population data for 2020
population_2020 = get_from_census("B01003", "2020")

# clean it up
population_2020 = population_2020\
        [['zip code tabulation area', 'B01003_001E']]\
        .rename(columns={'zip code tabulation area': 'zip'})

population_2020

,zip,B01003_001E
1,29590,3543
2,93306,74296
3,93660,4082
4,93110,15777
5,93212,22596
...,...,...
33116,16623,552
33117,16627,2118
33118,16634,315
33119,16640,707


In [ ]:
#rename columns
population_2020.rename(columns={'B01003_001E': 'pop_2020'}, inplace=True)
population_2020

,zip,pop_2020
1,29590,3543
2,93306,74296
3,93660,4082
4,93110,15777
5,93212,22596
...,...,...
33116,16623,552
33117,16627,2118
33118,16634,315
33119,16640,707


In [ ]:
population = pd.merge(population_2020, population_2021, on='zip', how='outer')
population

,zip,pop_2020,pop_2021
0,29590,3543,3560
1,93306,74296,74518
2,93660,4082,3826
3,93110,15777,16345
4,93212,22596,23148
...,...,...,...
33966,99635,NaN,107
33967,99675,NaN,58
33968,99707,NaN,0
33969,99725,NaN,611


In [ ]:
# merge population and median income data into one dataframe

census_data = pd.merge(population, median_income, on='zip', how='inner')
census_data


,zip,pop_2020,pop_2021,med_inc_2020,med_inc_2021
0,29590,3543,3560,30985,52679
1,93306,74296,74518,54450,60857
2,93660,4082,3826,39625,40000
3,93110,15777,16345,93264,99261
4,93212,22596,23148,42983,46312
...,...,...,...,...,...
33966,99635,NaN,107,NaN,-666666666
33967,99675,NaN,58,NaN,34375
33968,99707,NaN,0,NaN,-666666666
33969,99725,NaN,611,NaN,45931


Merge with zip code csv

In [ ]:
# read csv
zip_codes = pd.read_csv('nyc_zip_codes_cleaned.csv')
zip_codes

FileNotFoundError: [Errno 2] No such file or directory: 'nyc_zip_codes_cleaned.csv'

In [ ]:
zip_codes.dtypes

zip         int64
borough    object
dtype: object

In [ ]:
census_data.dtypes


zip             object
pop_2020        object
pop_2021        object
med_inc_2020    object
med_inc_2021    object
dtype: object

In [ ]:
# convert zip codes to int

census_data['zip'] = census_data['zip'].astype(int)
census_data.dtypes


zip              int64
pop_2020        object
pop_2021        object
med_inc_2020    object
med_inc_2021    object
dtype: object

In [ ]:
# left join zip codes with census data

census = pd.merge(zip_codes, census_data, on='zip', how='left')
census

,zip,borough,pop_2020,pop_2021,med_inc_2020,med_inc_2021
0,10001,Manhattan,25026,26966,96787,101409
1,10002,Manhattan,74363,76807,35607,37093
2,10003,Manhattan,54671,54447,129981,137533
3,10004,Manhattan,3310,4795,204949,216017
4,10005,Manhattan,8664,8637,184681,197188
...,...,...,...,...,...,...
475,10309,Staten,33896,35832,102730,107500
476,10310,Staten,24168,25976,86895,96161
477,10311,Staten,0,0,-666666666,-666666666
478,10312,Staten,61114,63935,96785,100875


In [ ]:
# data for 2022 is not available yet, so we will use 2021 data for 2022
census['pop_2022'] = census['pop_2021']
census['med_inc_2022'] = census['med_inc_2021']
census

,zip,borough,pop_2020,pop_2021,med_inc_2020,med_inc_2021,pop_2022,med_inc_2022
0,10001,Manhattan,25026,26966,96787,101409,26966,101409
1,10002,Manhattan,74363,76807,35607,37093,76807,37093
2,10003,Manhattan,54671,54447,129981,137533,54447,137533
3,10004,Manhattan,3310,4795,204949,216017,4795,216017
4,10005,Manhattan,8664,8637,184681,197188,8637,197188
...,...,...,...,...,...,...,...,...
475,10309,Staten,33896,35832,102730,107500,35832,107500
476,10310,Staten,24168,25976,86895,96161,25976,96161
477,10311,Staten,0,0,-666666666,-666666666,0,-666666666
478,10312,Staten,61114,63935,96785,100875,63935,100875


In [ ]:
# how many unique zip codes are there?

census['zip'].nunique()

240

In [ ]:
# save to csv
# census.to_csv('census_data.csv', index=False)

### Tract level data from Census API

In [ ]:
# !pip install censusgeocode
# ! pip install requests-cache
# ! brew install redis
# ! brew services start redis
# ! pip install redis

In [1]:
import json
from census import Census
from us import states
import glob
from pprint import pprint
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
import requests_cache

requests_cache.install_cache('./censusgeo_cache', backend='redis')
# requests.get('https://httpbin.org/delay/1')


In [3]:
# df = pd.read_csv('address_level.csv')
# df


In [4]:
# save as parquet

# df.to_parquet('address_level.parquet', index=False)

In [2]:
df_address = pd.read_parquet('address_level.parquet')
df_address

,year,borough,zip,incident_address,lat,long,num_complaints
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
106127,2022,STATEN ISLAND,10314,81 CHURCH AVENUE,40.592844,-74.185280,1
106128,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587616,-74.165297,1
106129,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587651,-74.165359,5
106130,2022,STATEN ISLAND,10314,98 NOSTRAND AVENUE,40.606914,-74.169706,1


In [3]:
# only keep the columns where borough is Bronx

df_borough_bronx = df_address[df_address['borough'] == 'BRONX']
df_borough_bronx

,year,borough,zip,incident_address,lat,long,num_complaints
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
74282,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1
74283,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1
74284,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6
74285,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1


In [47]:
df_borough_bronx.reset_index(inplace=True)
df_borough_bronx

,index,year,borough,zip,incident_address,lat,long,num_complaints
0,0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...,...
27039,74282,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1
27040,74283,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1
27041,74284,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6
27042,74285,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1


In [46]:
census_geos_df

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,0,360050183013003,+40.8303034,3003,0,36,3003,210701006017485,BK,...,018301,-073.9207286,3,1647,0,-073.9207286,G5040,L,U,005
1,,1114,360050063013000,+40.8245704,3000,0,36,3000,210701006029989,BK,...,006301,-073.9272224,3,21344,350,-073.9272224,G5040,L,U,005
2,,295,360050063014004,+40.8208371,4004,0,36,4004,210701006029630,BK,...,006301,-073.9269296,4,7425,105,-073.9269296,G5040,L,U,005
3,,559,360050183011003,+40.8270688,1003,0,36,1003,210701006017467,BK,...,018301,-073.9215858,1,11274,419,-073.9215858,G5040,L,U,005
4,,1073,360050183011000,+40.8278863,1000,0,36,1000,210701006027054,BK,...,018301,-073.9211491,1,13780,459,-073.9211491,G5040,L,U,005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,,1028,360050462033016,+40.8798570,3016,0,36,3016,210701006027019,BK,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005
27040,,815,360050462071003,+40.8771884,1003,0,36,1003,210701006029142,BK,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27041,,815,360050462071003,+40.8771884,1003,0,36,1003,210701006029142,BK,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27042,,1028,360050462033016,+40.8798570,3016,0,36,3016,210701006027019,BK,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005


In [4]:
# census block data for 2021
base_url_2021 = 'https://api.census.gov/data/2021/acs/acs5'
params = {'get': 'NAME,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B03002_002E,B03002_001E,B17017_002E,B17017_001E',
          'for': 'tract:*',
        #   code for the five boroughs
          'in': 'state:36 county:005,047,061,081,085'}

response = requests.get(base_url_2021, params=params)

# Convert the response to a Pandas dataframe
tract_2021 = pd.DataFrame(response.json()[1:], columns=response.json()[0])

print(tract_2021.head())


                                         NAME B02001_001E B02001_002E  \
0      Census Tract 1, Bronx County, New York        6661        2680   
1      Census Tract 2, Bronx County, New York        4453        1235   
2      Census Tract 4, Bronx County, New York        6000        1809   
3     Census Tract 16, Bronx County, New York        6038         996   
4  Census Tract 19.01, Bronx County, New York        2168         664   

  B02001_003E B02001_004E B02001_005E B03002_002E B03002_001E B17017_002E  \
0        3272          39         177        4495        6661           0   
1        1262           0         160        1273        4453         278   
2        1472          18         177        1614        6000         107   
3        2322         213          65        2674        6038         526   
4         868          27           0        1013        2168         251   

  B17017_001E state county   tract  
0           0    36    005  000100  
1        1392    36    0

In [ ]:
# B02001_001E - Total_race
# B02001_002E - White alone
# B02001_003E - Black or African American alone
# B02001_004E - American Indian and Alaska Native alone
# B02001_005E - Asian alone
# B03002_002E - Not Hispanic or Latino
# B03002_001E - Total_in_hispanic
# B17017_002E households_below_poverty
# B17017_001E - total_households

In [6]:
# rename columns

tract_2021 = tract_2021.rename(columns={
    'B02001_001E': 'total_race',
    'B02001_002E': 'white',
    'B02001_003E': 'black',
    'B02001_005E': 'asian',
    'B03002_001E': 'total_in_hispanic',
    'B03002_002E': 'not_hispanic',
    'B17017_001E': 'total_households',
    'B17017_002E': 'households_below_poverty'
    })

tract_2021.head(1)


,NAME,total_race,white,black,B02001_004E,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract
0,"Census Tract 1, Bronx County, New York",6661,2680,3272,39,177,4495,6661,0,0,36,005,000100


In [7]:
tract_2021.dtypes

NAME                        object
total_race                  object
white                       object
black                       object
B02001_004E                 object
asian                       object
not_hispanic                object
total_in_hispanic           object
households_below_poverty    object
total_households            object
state                       object
county                      object
tract                       object
dtype: object

In [13]:
# convert columns to int

tract_2021['total_race'] = tract_2021['total_race'].astype(int)
tract_2021['white'] = tract_2021['white'].astype(int)
tract_2021['black'] = tract_2021['black'].astype(int)
tract_2021['asian'] = tract_2021['asian'].astype(int)
tract_2021['total_in_hispanic'] = tract_2021['total_in_hispanic'].astype(int)
tract_2021['not_hispanic'] = tract_2021['not_hispanic'].astype(int)
tract_2021['total_households'] = tract_2021['total_households'].astype(int)
tract_2021['households_below_poverty'] = tract_2021['households_below_poverty'].astype(int)
tract_2021['tract'] = tract_2021['tract'].astype(int)
tract_2021.head(1)





,NAME,total_race,white,black,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract
0,"Census Tract 1, Bronx County, New York",6661,2680,3272,177,4495,6661,0,0,36,005,100


In [14]:
tract_2021.dtypes

NAME                        object
total_race                   int64
white                        int64
black                        int64
asian                        int64
not_hispanic                 int64
total_in_hispanic            int64
households_below_poverty     int64
total_households             int64
state                       object
county                      object
tract                        int64
dtype: object

In [16]:
# new columns for percentage 

tract_2021['pct_white'] = tract_2021['white'] / tract_2021['total_race'] * 100
tract_2021['pct_black'] = tract_2021['black'] / tract_2021['total_race'] * 100
tract_2021['pct_asian'] = tract_2021['asian'] / tract_2021['total_race'] * 100
tract_2021['hispanic'] = tract_2021['total_in_hispanic'] - tract_2021['not_hispanic'] 
tract_2021['pct_hispanic'] = tract_2021['hispanic'] / tract_2021['total_in_hispanic'] * 100
tract_2021['pct_below_poverty'] = tract_2021['households_below_poverty'] / tract_2021['total_households'] * 100
tract_2021.head(5)


,NAME,total_race,white,black,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,"Census Tract 1, Bronx County, New York",6661,2680,3272,177,4495,6661,0,0,36,005,100,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,"Census Tract 2, Bronx County, New York",4453,1235,1262,160,1273,4453,278,1392,36,005,200,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,"Census Tract 4, Bronx County, New York",6000,1809,1472,177,1614,6000,107,2199,36,005,400,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,"Census Tract 16, Bronx County, New York",6038,996,2322,65,2674,6038,526,2187,36,005,1600,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,"Census Tract 19.01, Bronx County, New York",2168,664,868,0,1013,2168,251,885,36,005,1901,30.627306,40.036900,0.000000,1155,53.274908,28.361582


### Get tract level data from censusgeocode

In [7]:
import pandas as pd
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df_borough_bronx['lat']
        longitudes = df_borough_bronx['long']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df_borough_bronx)))

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df_borough_bronx['lat'], df_borough_bronx['long']) 
census_geos_df


  0%|          | 0/27044 [00:00<?, ?it/s]

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,0,360050183013003,+40.8303034,3003,0,36,3003,210701006017485,BK,...,018301,-073.9207286,3,1647,0,-073.9207286,G5040,L,U,005
1,,1114,360050063013000,+40.8245704,3000,0,36,3000,210701006029989,BK,...,006301,-073.9272224,3,21344,350,-073.9272224,G5040,L,U,005
2,,295,360050063014004,+40.8208371,4004,0,36,4004,210701006029630,BK,...,006301,-073.9269296,4,7425,105,-073.9269296,G5040,L,U,005
3,,559,360050183011003,+40.8270688,1003,0,36,1003,210701006017467,BK,...,018301,-073.9215858,1,11274,419,-073.9215858,G5040,L,U,005
4,,1073,360050183011000,+40.8278863,1000,0,36,1000,210701006027054,BK,...,018301,-073.9211491,1,13780,459,-073.9211491,G5040,L,U,005


In [11]:
census_geos_df.to_parquet('census_geos_df.parquet', index=False)

In [41]:
census_geos_df = pd.read_parquet('census_geos_df.parquet')
census_geos_df

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,0,360050183013003,+40.8303034,3003,0,36,3003,210701006017485,BK,...,018301,-073.9207286,3,1647,0,-073.9207286,G5040,L,U,005
1,,1114,360050063013000,+40.8245704,3000,0,36,3000,210701006029989,BK,...,006301,-073.9272224,3,21344,350,-073.9272224,G5040,L,U,005
2,,295,360050063014004,+40.8208371,4004,0,36,4004,210701006029630,BK,...,006301,-073.9269296,4,7425,105,-073.9269296,G5040,L,U,005
3,,559,360050183011003,+40.8270688,1003,0,36,1003,210701006017467,BK,...,018301,-073.9215858,1,11274,419,-073.9215858,G5040,L,U,005
4,,1073,360050183011000,+40.8278863,1000,0,36,1000,210701006027054,BK,...,018301,-073.9211491,1,13780,459,-073.9211491,G5040,L,U,005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,,1028,360050462033016,+40.8798570,3016,0,36,3016,210701006027019,BK,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005
27040,,815,360050462071003,+40.8771884,1003,0,36,1003,210701006029142,BK,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27041,,815,360050462071003,+40.8771884,1003,0,36,1003,210701006029142,BK,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27042,,1028,360050462033016,+40.8798570,3016,0,36,3016,210701006027019,BK,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005


In [50]:
bronx_census = pd.concat([df_borough_bronx, census_geos_df], axis=1)
bronx_census.to_parquet('bronx_census_combined.parquet', index=False)
bronx_census

,index,year,borough,zip,incident_address,lat,long,num_complaints,SUFFIX,POP100,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,,0,...,018301,-073.9207286,3,1647,0,-073.9207286,G5040,L,U,005
1,1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108,,1114,...,006301,-073.9272224,3,21344,350,-073.9272224,G5040,L,U,005
2,2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2,,295,...,006301,-073.9269296,4,7425,105,-073.9269296,G5040,L,U,005
3,3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2,,559,...,018301,-073.9215858,1,11274,419,-073.9215858,G5040,L,U,005
4,4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2,,1073,...,018301,-073.9211491,1,13780,459,-073.9211491,G5040,L,U,005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,74282,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1,,1028,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005
27040,74283,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1,,815,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27041,74284,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6,,815,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27042,74285,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1,,1028,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005


In [51]:
bronx_census = bronx_census.rename(columns={'TRACT': 'tract'})
bronx_census

,index,year,borough,zip,incident_address,lat,long,num_complaints,SUFFIX,POP100,...,tract,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,,0,...,018301,-073.9207286,3,1647,0,-073.9207286,G5040,L,U,005
1,1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108,,1114,...,006301,-073.9272224,3,21344,350,-073.9272224,G5040,L,U,005
2,2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2,,295,...,006301,-073.9269296,4,7425,105,-073.9269296,G5040,L,U,005
3,3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2,,559,...,018301,-073.9215858,1,11274,419,-073.9215858,G5040,L,U,005
4,4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2,,1073,...,018301,-073.9211491,1,13780,459,-073.9211491,G5040,L,U,005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,74282,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1,,1028,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005
27040,74283,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1,,815,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27041,74284,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6,,815,...,046207,-073.8330987,1,22558,419,-073.8330987,G5040,L,U,005
27042,74285,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1,,1028,...,046203,-073.8293674,3,105186,421,-073.8293674,G5040,L,U,005


In [52]:
# save as csv

# bronx_census.to_csv('bronx_census.csv', index=False)

In [53]:
# check columns

bronx_census.columns

Index(['index', 'year', 'borough', 'zip', 'incident_address', 'lat', 'long',
       'num_complaints', 'SUFFIX', 'POP100', 'GEOID', 'CENTLAT', 'BLOCK',
       'AREAWATER', 'STATE', 'BASENAME', 'OID', 'LSADC', 'INTPTLAT',
       'FUNCSTAT', 'NAME', 'OBJECTID', 'tract', 'CENTLON', 'BLKGRP',
       'AREALAND', 'HU100', 'INTPTLON', 'MTFCC', 'LWBLKTYP', 'UR', 'COUNTY'],
      dtype='object')

In [54]:
# remove columns 

bronx_census = bronx_census.drop(columns=['SUFFIX', 'POP100', 'GEOID', 'CENTLAT', 'AREAWATER', 'STATE', 'BASENAME', 'OID', 'LSADC', 'INTPTLAT', 'FUNCSTAT', 'OBJECTID', 'CENTLON', 'BLKGRP', 'AREALAND', 'HU100', 'INTPTLON', 'MTFCC', 'LWBLKTYP', 'UR', 'COUNTY'])
bronx_census


,year,borough,zip,incident_address,lat,long,num_complaints,BLOCK,NAME,tract
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,3003,Block 3003,018301
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108,3000,Block 3000,006301
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2,4004,Block 4004,006301
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2,1003,Block 1003,018301
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2,1000,Block 1000,018301
...,...,...,...,...,...,...,...,...,...,...
27039,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1,3016,Block 3016,046203
27040,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1,1003,Block 1003,046207
27041,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6,1003,Block 1003,046207
27042,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1,3016,Block 3016,046203


In [55]:
bronx_census = bronx_census.drop(columns=['NAME', 'BLOCK'])
bronx_census

,year,borough,zip,incident_address,lat,long,num_complaints,tract
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,018301
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108,006301
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2,006301
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2,018301
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2,018301
...,...,...,...,...,...,...,...,...
27039,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1,046203
27040,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1,046207
27041,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6,046207
27042,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1,046203


In [168]:
# arrange in descending order by num_complaints

bronx_census.sort_values(by='num_complaints', ascending=False)


,year,borough,zip,incident_address,lat,long,num_complaints,tract
10143,2021,BRONX,10457,2176 TIEBOUT AVENUE,40.854439,-73.898423,3707,38303
19938,2022,BRONX,10457,2176 TIEBOUT AVENUE,40.854439,-73.898423,3310,38303
2235,2020,BRONX,10457,2176 TIEBOUT AVENUE,40.854439,-73.898423,3242,38303
19780,2022,BRONX,10457,2000 ANTHONY AVENUE,40.851443,-73.902863,296,38100
19856,2022,BRONX,10457,2082 HUGHES AVENUE,40.848514,-73.890697,247,37300
...,...,...,...,...,...,...,...,...
9822,2021,BRONX,10457,1685 TOPPING AVENUE,40.843799,-73.905852,1,22902
19675,2022,BRONX,10457,1800 MONROE AVENUE,40.846862,-73.906470,1,22901
9816,2021,BRONX,10457,1668 GRAND CONCOURSE,40.843680,-73.911758,1,22703
19678,2022,BRONX,10457,1805 CLINTON AVENUE,40.842195,-73.892612,1,36902


In [ ]:
# Notes: 2176 TIEBOUT AVENUE - tract 38303 - 3707 complaints in 2021

In [169]:
# sum of num_complaints in 2021 from tract 38303

bronx_census[bronx_census['tract'] == 38303]['num_complaints'].sum()

10499

In [56]:
bronx_census.dtypes

year                  int64
borough              object
zip                   int64
incident_address     object
lat                 float64
long                float64
num_complaints        int64
tract                object
dtype: object

In [57]:
bronx_census['tract'] = bronx_census['tract'].astype(int)
bronx_census.dtypes

year                  int64
borough              object
zip                   int64
incident_address     object
lat                 float64
long                float64
num_complaints        int64
tract                 int64
dtype: object

In [58]:
# does tract column have any null values? - sanity check

bronx_census['tract'].isnull().sum()

0

In [60]:
bronx_census

,year,borough,zip,incident_address,lat,long,num_complaints,tract
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,18301
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108,6301
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2,6301
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2,18301
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2,18301
...,...,...,...,...,...,...,...,...
27039,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1,46203
27040,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1,46207
27041,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6,46207
27042,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1,46203


In [118]:
# keep only borough BRONX and year 2021 

bronx_census_2021 = bronx_census[(bronx_census['borough'] == 'BRONX') & (bronx_census['year'] == 2021)]
bronx_census_2021

,year,borough,zip,incident_address,lat,long,num_complaints,tract
7119,2021,BRONX,10451,1000 GRAND CONCOURSE,40.829790,-73.921074,4,18301
7120,2021,BRONX,10451,1000 GRAND CONCOURSE,40.829818,-73.921060,2,18301
7121,2021,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,18301
7122,2021,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,92,6301
7123,2021,BRONX,10451,118 EAST 138 STREET,40.813438,-73.930732,1,5100
...,...,...,...,...,...,...,...,...
16378,2021,BRONX,11106,35-34 CRESCENT STREET,40.759170,-73.932507,1,4700
16379,2021,BRONX,11370,11-11 HAZEN STREET,40.793378,-73.884139,1,100
16380,2021,BRONX,11370,11-11 HAZEN STREET,40.793471,-73.884139,1,100
16381,2021,BRONX,11370,16-16 HAZEN STREET,40.790301,-73.884412,1,100


In [121]:
# drop columns

bronx_small = bronx_census_2021.drop(columns=['borough', 'year', 'lat', 'long', 'incident_address', 'zip'])
bronx_small

,num_complaints,tract
7119,4,18301
7120,2,18301
7121,1,18301
7122,92,6301
7123,1,5100
...,...,...
16378,1,4700
16379,1,100
16380,1,100
16381,1,100


In [123]:
# reset index 

bronx_small.reset_index(inplace=True)
bronx_small

,level_0,index,num_complaints,tract
0,0,7119,4,18301
1,1,7120,2,18301
2,2,7121,1,18301
3,3,7122,92,6301
4,4,7123,1,5100
...,...,...,...,...
9259,9259,16378,1,4700
9260,9260,16379,1,100
9261,9261,16380,1,100
9262,9262,16381,1,100


In [125]:
bronx_small = bronx_small.drop(columns=['level_0'])
bronx_small

,index,num_complaints,tract
0,7119,4,18301
1,7120,2,18301
2,7121,1,18301
3,7122,92,6301
4,7123,1,5100
...,...,...,...
9259,16378,1,4700
9260,16379,1,100
9261,16380,1,100
9262,16381,1,100


In [128]:
# group by tract and calculate the sum of num_complaints

bronx_small = bronx_small.groupby('tract')['num_complaints'].sum()
bronx_small

tract
100        3
200      108
400       46
1600     110
1901     189
        ... 
46208     74
46209     31
48401     28
48402      2
51601     41
Name: num_complaints, Length: 366, dtype: int64

In [129]:
bronx_small = bronx_small.to_frame()
bronx_small

,num_complaints
tract,
100,3
200,108
400,46
1600,110
1901,189
...,...
46208,74
46209,31
48401,28


In [130]:
# rename num_complaints to complaints 

bronx_small = bronx_small.rename(columns={'num_complaints': 'complaints'})
bronx_small

,complaints
tract,
100,3
200,108
400,46
1600,110
1901,189
...,...
46208,74
46209,31
48401,28


In [131]:
bronx_small.columns

Index(['complaints'], dtype='object')

In [134]:
# convert index to column

bronx_small.reset_index(inplace=True)
bronx_small

,tract,complaints
0,100,3
1,200,108
2,400,46
3,1600,110
4,1901,189
...,...,...
361,46208,74
362,46209,31
363,48401,28
364,48402,2


In [61]:
tract_2021

,NAME,total_race,white,black,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,"Census Tract 1, Bronx County, New York",6661,2680,3272,177,4495,6661,0,0,36,005,100,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,"Census Tract 2, Bronx County, New York",4453,1235,1262,160,1273,4453,278,1392,36,005,200,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,"Census Tract 4, Bronx County, New York",6000,1809,1472,177,1614,6000,107,2199,36,005,400,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,"Census Tract 16, Bronx County, New York",6038,996,2322,65,2674,6038,526,2187,36,005,1600,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,"Census Tract 19.01, Bronx County, New York",2168,664,868,0,1013,2168,251,885,36,005,1901,30.627306,40.036900,0.000000,1155,53.274908,28.361582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322,"Census Tract 303.02, Richmond County, New York",6663,2765,1486,678,4078,6663,238,2061,36,085,30302,41.497824,22.302266,10.175597,2585,38.796338,11.547792
2323,"Census Tract 319.01, Richmond County, New York",3422,695,1661,602,2479,3422,504,1148,36,085,31901,20.309760,48.538866,17.592051,943,27.556984,43.902439
2324,"Census Tract 319.02, Richmond County, New York",5298,1363,3045,227,3481,5298,408,1569,36,085,31902,25.726689,57.474519,4.284636,1817,34.295961,26.003824
2325,"Census Tract 323, Richmond County, New York",1105,338,431,33,776,1105,61,420,36,085,32300,30.588235,39.004525,2.986425,329,29.773756,14.523810


In [76]:
# sanity check - why is the merge not working?

tract_2021[tract_2021['tract'] == 15300]

,NAME,total_race,white,black,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
96,"Census Tract 153, Bronx County, New York",4226,270,2341,90,1940,4226,622,1741,36,005,15300,6.389020,55.395173,2.129673,2286,54.093706,35.726594
490,"Census Tract 153, Kings County, New York",2830,2148,106,187,2410,2830,74,1138,36,047,15300,75.901060,3.745583,6.607774,420,14.840989,6.502636
1599,"Census Tract 153, Queens County, New York",1884,1145,47,445,1620,1884,94,779,36,081,15300,60.774947,2.494692,23.619958,264,14.012739,12.066752


In [1]:
# tract numbers are not unique 

In [79]:
# keep rows that contain 'Bronx County' in NAME column

tract_2021 = tract_2021[tract_2021['NAME'].str.contains('Bronx County')]
tract_2021

,NAME,total_race,white,black,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,"Census Tract 1, Bronx County, New York",6661,2680,3272,177,4495,6661,0,0,36,005,100,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,"Census Tract 2, Bronx County, New York",4453,1235,1262,160,1273,4453,278,1392,36,005,200,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,"Census Tract 4, Bronx County, New York",6000,1809,1472,177,1614,6000,107,2199,36,005,400,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,"Census Tract 16, Bronx County, New York",6038,996,2322,65,2674,6038,526,2187,36,005,1600,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,"Census Tract 19.01, Bronx County, New York",2168,664,868,0,1013,2168,251,885,36,005,1901,30.627306,40.036900,0.000000,1155,53.274908,28.361582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,"Census Tract 484.01, Bronx County, New York",4213,239,3461,0,3725,4213,87,1267,36,005,48401,5.672917,82.150487,0.000000,488,11.583195,6.866614
357,"Census Tract 484.02, Bronx County, New York",0,0,0,0,0,0,0,0,36,005,48402,NaN,NaN,NaN,0,NaN,NaN
358,"Census Tract 504, Bronx County, New York",0,0,0,0,0,0,0,0,36,005,50400,NaN,NaN,NaN,0,NaN,NaN
359,"Census Tract 516.01, Bronx County, New York",4417,3399,31,87,3230,4417,125,1958,36,005,51601,76.952683,0.701834,1.969663,1187,26.873444,6.384065


In [82]:

bronx_data = pd.merge(bronx_census, tract_2021, on='tract', how='left')
bronx_data



,year,borough,zip,incident_address,lat,long,num_complaints,tract,NAME,total_race,...,households_below_poverty,total_households,state,county,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1,18301,"Census Tract 183.01, Bronx County, New York",4657.0,...,394.0,2072.0,36,005,20.313507,31.694224,2.426455,2562.0,55.013957,19.015444
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108,6301,"Census Tract 63.01, Bronx County, New York",5521.0,...,540.0,2110.0,36,005,24.904909,35.609491,2.191632,3059.0,55.406629,25.592417
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2,6301,"Census Tract 63.01, Bronx County, New York",5521.0,...,540.0,2110.0,36,005,24.904909,35.609491,2.191632,3059.0,55.406629,25.592417
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2,18301,"Census Tract 183.01, Bronx County, New York",4657.0,...,394.0,2072.0,36,005,20.313507,31.694224,2.426455,2562.0,55.013957,19.015444
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2,18301,"Census Tract 183.01, Bronx County, New York",4657.0,...,394.0,2072.0,36,005,20.313507,31.694224,2.426455,2562.0,55.013957,19.015444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27039,2022,BRONX,10475,789 CO OP CITY BOULEVARD,40.878583,-73.827933,1,46203,"Census Tract 462.03, Bronx County, New York",7733.0,...,396.0,3245.0,36,005,17.380059,62.200957,0.387948,2595.0,33.557481,12.203390
27040,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.876931,-73.833698,1,46207,"Census Tract 462.07, Bronx County, New York",6351.0,...,524.0,2968.0,36,005,7.920013,62.068966,4.204062,2038.0,32.089435,17.654987
27041,2022,BRONX,10475,920 BAYCHESTER AVENUE,40.877178,-73.833625,6,46207,"Census Tract 462.07, Bronx County, New York",6351.0,...,524.0,2968.0,36,005,7.920013,62.068966,4.204062,2038.0,32.089435,17.654987
27042,2022,BRONX,10475,920 CO OP CITY BOULEVARD,40.878471,-73.830429,1,46203,"Census Tract 462.03, Bronx County, New York",7733.0,...,396.0,3245.0,36,005,17.380059,62.200957,0.387948,2595.0,33.557481,12.203390


In [83]:
# save as csv

bronx_data.to_csv('bronx_data.csv', index=False)

#### Get tract level population to normalize complaints

(Forgot to fetch population earlier)

In [85]:
# census block data for 2021
tract_pop_2021 = 'https://api.census.gov/data/2021/acs/acs5'
params = {'get': 'NAME,B01003_001E',
          'for': 'tract:*',
        #   code for the five boroughs
          'in': 'state:36 county:005'}

response = requests.get(tract_pop_2021, params=params)

# Convert the response to a Pandas dataframe
tract_pop_2021 = pd.DataFrame(response.json()[1:], columns=response.json()[0])

print(tract_pop_2021)

                                            NAME B01003_001E state county  \
0         Census Tract 1, Bronx County, New York        6661    36    005   
1         Census Tract 2, Bronx County, New York        4453    36    005   
2         Census Tract 4, Bronx County, New York        6000    36    005   
3        Census Tract 16, Bronx County, New York        6038    36    005   
4     Census Tract 19.01, Bronx County, New York        2168    36    005   
..                                           ...         ...   ...    ...   
356  Census Tract 484.01, Bronx County, New York        4213    36    005   
357  Census Tract 484.02, Bronx County, New York           0    36    005   
358     Census Tract 504, Bronx County, New York           0    36    005   
359  Census Tract 516.01, Bronx County, New York        4417    36    005   
360  Census Tract 516.02, Bronx County, New York           0    36    005   

      tract  
0    000100  
1    000200  
2    000400  
3    001600  
4    

In [90]:
tract_pop_2021.dtypes

NAME           object
B01003_001E    object
state          object
county         object
tract          object
dtype: object

In [91]:
# convert tract column to int

tract_pop_2021['tract'] = tract_pop_2021['tract'].astype(int)
tract_pop_2021.dtypes


NAME           object
B01003_001E    object
state          object
county         object
tract           int64
dtype: object

In [97]:
# rename columns

tract_pop_2021 = tract_pop_2021.rename(columns={'pop': 'pop_2021'})
tract_pop_2021

,NAME,pop_2021,state,county,tract
0,"Census Tract 1, Bronx County, New York",6661,36,005,100
1,"Census Tract 2, Bronx County, New York",4453,36,005,200
2,"Census Tract 4, Bronx County, New York",6000,36,005,400
3,"Census Tract 16, Bronx County, New York",6038,36,005,1600
4,"Census Tract 19.01, Bronx County, New York",2168,36,005,1901
...,...,...,...,...,...
356,"Census Tract 484.01, Bronx County, New York",4213,36,005,48401
357,"Census Tract 484.02, Bronx County, New York",0,36,005,48402
358,"Census Tract 504, Bronx County, New York",0,36,005,50400
359,"Census Tract 516.01, Bronx County, New York",4417,36,005,51601


In [137]:
bronx_small


,tract,complaints
0,100,3
1,200,108
2,400,46
3,1600,110
4,1901,189
...,...,...
361,46208,74
362,46209,31
363,48401,28
364,48402,2


In [139]:

bronx_2021_data = pd.merge(tract_pop_2021, bronx_small, on='tract', how='left')
bronx_2021_data

,NAME,pop_2021,state,county,tract,complaints
0,"Census Tract 1, Bronx County, New York",6661,36,005,100,3.0
1,"Census Tract 2, Bronx County, New York",4453,36,005,200,108.0
2,"Census Tract 4, Bronx County, New York",6000,36,005,400,46.0
3,"Census Tract 16, Bronx County, New York",6038,36,005,1600,110.0
4,"Census Tract 19.01, Bronx County, New York",2168,36,005,1901,189.0
...,...,...,...,...,...,...
356,"Census Tract 484.01, Bronx County, New York",4213,36,005,48401,28.0
357,"Census Tract 484.02, Bronx County, New York",0,36,005,48402,2.0
358,"Census Tract 504, Bronx County, New York",0,36,005,50400,NaN
359,"Census Tract 516.01, Bronx County, New York",4417,36,005,51601,41.0


In [141]:
# remove rows where pop_2021 is 0

bronx_2021_data = bronx_2021_data[bronx_2021_data['pop_2021'] != '0']
bronx_2021_data

,NAME,pop_2021,state,county,tract,complaints
0,"Census Tract 1, Bronx County, New York",6661,36,005,100,3.0
1,"Census Tract 2, Bronx County, New York",4453,36,005,200,108.0
2,"Census Tract 4, Bronx County, New York",6000,36,005,400,46.0
3,"Census Tract 16, Bronx County, New York",6038,36,005,1600,110.0
4,"Census Tract 19.01, Bronx County, New York",2168,36,005,1901,189.0
...,...,...,...,...,...,...
353,"Census Tract 462.07, Bronx County, New York",6351,36,005,46207,20.0
354,"Census Tract 462.08, Bronx County, New York",5879,36,005,46208,74.0
355,"Census Tract 462.09, Bronx County, New York",5039,36,005,46209,31.0
356,"Census Tract 484.01, Bronx County, New York",4213,36,005,48401,28.0


In [145]:
bronx_2021_data.dtypes

NAME           object
pop_2021        int64
state          object
county         object
tract           int64
complaints    float64
dtype: object

In [149]:
# create a new column called complaints_norm which is complaints/pop_2021 * 1000

bronx_2021_data['complaints_norm'] = bronx_2021_data['complaints'] / bronx_2021_data['pop_2021'] * 1000
bronx_2021_data



/var/folders/3x/_wb_w3ys63j7vqmgfcp3xx_w0000gn/T/ipykernel_38243/1958105242.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bronx_2021_data['complaints_norm'] = bronx_2021_data['complaints'] / bronx_2021_data['pop_2021'] * 1000


,NAME,pop_2021,state,county,tract,complaints,complaints_norm
0,"Census Tract 1, Bronx County, New York",6661,36,005,100,3.0,0.450383
1,"Census Tract 2, Bronx County, New York",4453,36,005,200,108.0,24.253312
2,"Census Tract 4, Bronx County, New York",6000,36,005,400,46.0,7.666667
3,"Census Tract 16, Bronx County, New York",6038,36,005,1600,110.0,18.217953
4,"Census Tract 19.01, Bronx County, New York",2168,36,005,1901,189.0,87.177122
...,...,...,...,...,...,...,...
353,"Census Tract 462.07, Bronx County, New York",6351,36,005,46207,20.0,3.149110
354,"Census Tract 462.08, Bronx County, New York",5879,36,005,46208,74.0,12.587175
355,"Census Tract 462.09, Bronx County, New York",5039,36,005,46209,31.0,6.152014
356,"Census Tract 484.01, Bronx County, New York",4213,36,005,48401,28.0,6.646095


In [150]:
# remove columns 

bronx_2021_data = bronx_2021_data.drop(columns=['NAME', 'state', 'county'])
bronx_2021_data

,pop_2021,tract,complaints,complaints_norm
0,6661,100,3.0,0.450383
1,4453,200,108.0,24.253312
2,6000,400,46.0,7.666667
3,6038,1600,110.0,18.217953
4,2168,1901,189.0,87.177122
...,...,...,...,...
353,6351,46207,20.0,3.149110
354,5879,46208,74.0,12.587175
355,5039,46209,31.0,6.152014
356,4213,48401,28.0,6.646095


In [151]:
tract_2021

,NAME,total_race,white,black,asian,not_hispanic,total_in_hispanic,households_below_poverty,total_households,state,county,tract,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,"Census Tract 1, Bronx County, New York",6661,2680,3272,177,4495,6661,0,0,36,005,100,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,"Census Tract 2, Bronx County, New York",4453,1235,1262,160,1273,4453,278,1392,36,005,200,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,"Census Tract 4, Bronx County, New York",6000,1809,1472,177,1614,6000,107,2199,36,005,400,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,"Census Tract 16, Bronx County, New York",6038,996,2322,65,2674,6038,526,2187,36,005,1600,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,"Census Tract 19.01, Bronx County, New York",2168,664,868,0,1013,2168,251,885,36,005,1901,30.627306,40.036900,0.000000,1155,53.274908,28.361582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,"Census Tract 484.01, Bronx County, New York",4213,239,3461,0,3725,4213,87,1267,36,005,48401,5.672917,82.150487,0.000000,488,11.583195,6.866614
357,"Census Tract 484.02, Bronx County, New York",0,0,0,0,0,0,0,0,36,005,48402,NaN,NaN,NaN,0,NaN,NaN
358,"Census Tract 504, Bronx County, New York",0,0,0,0,0,0,0,0,36,005,50400,NaN,NaN,NaN,0,NaN,NaN
359,"Census Tract 516.01, Bronx County, New York",4417,3399,31,87,3230,4417,125,1958,36,005,51601,76.952683,0.701834,1.969663,1187,26.873444,6.384065


In [157]:
# merge bronx_2021_data with tract_2021

bronx_2021_merged = pd.merge(bronx_2021_data, tract_2021, on='tract', how='left')
bronx_2021_merged

,pop_2021,tract,complaints,complaints_norm,NAME_x,total_race_x,white_x,black_x,asian_x,not_hispanic_x,...,households_below_poverty,total_households,state,county,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,6661,100,3.0,0.450383,"Census Tract 1, Bronx County, New York",6661,2680,3272,177,4495,...,0,0,36,005,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,4453,200,108.0,24.253312,"Census Tract 2, Bronx County, New York",4453,1235,1262,160,1273,...,278,1392,36,005,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,6000,400,46.0,7.666667,"Census Tract 4, Bronx County, New York",6000,1809,1472,177,1614,...,107,2199,36,005,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,6038,1600,110.0,18.217953,"Census Tract 16, Bronx County, New York",6038,996,2322,65,2674,...,526,2187,36,005,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,2168,1901,189.0,87.177122,"Census Tract 19.01, Bronx County, New York",2168,664,868,0,1013,...,251,885,36,005,30.627306,40.036900,0.000000,1155,53.274908,28.361582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,6351,46207,20.0,3.149110,"Census Tract 462.07, Bronx County, New York",6351,503,3942,267,4313,...,524,2968,36,005,7.920013,62.068966,4.204062,2038,32.089435,17.654987
343,5879,46208,74.0,12.587175,"Census Tract 462.08, Bronx County, New York",5879,575,3996,8,3966,...,239,1502,36,005,9.780575,67.970743,0.136078,1913,32.539548,15.912117
344,5039,46209,31.0,6.152014,"Census Tract 462.09, Bronx County, New York",5039,406,3245,4,2825,...,520,1488,36,005,8.057154,64.397698,0.079381,2214,43.937289,34.946237
345,4213,48401,28.0,6.646095,"Census Tract 484.01, Bronx County, New York",4213,239,3461,0,3725,...,87,1267,36,005,5.672917,82.150487,0.000000,488,11.583195,6.866614


In [158]:
bronx_2021_merged.columns

Index(['pop_2021', 'tract', 'complaints', 'complaints_norm', 'NAME_x',
       'total_race_x', 'white_x', 'black_x', 'asian_x', 'not_hispanic_x',
       'total_in_hispanic_x', 'households_below_poverty_x',
       'total_households_x', 'state_x', 'county_x', 'pct_white_x',
       'pct_black_x', 'pct_asian_x', 'hispanic_x', 'pct_hispanic_x',
       'pct_below_poverty_x', 'NAME_y', 'total_race_y', 'white_y', 'black_y',
       'asian_y', 'not_hispanic_y', 'total_in_hispanic_y',
       'households_below_poverty_y', 'total_households_y', 'state_y',
       'county_y', 'pct_white_y', 'pct_black_y', 'pct_asian_y', 'hispanic_y',
       'pct_hispanic_y', 'pct_below_poverty_y', 'NAME', 'total_race', 'white',
       'black', 'asian', 'not_hispanic', 'total_in_hispanic',
       'households_below_poverty', 'total_households', 'state', 'county',
       'pct_white', 'pct_black', 'pct_asian', 'hispanic', 'pct_hispanic',
       'pct_below_poverty'],
      dtype='object')

In [160]:
# remove columns 

bronx_2021_merged = bronx_2021_merged.drop(columns=['NAME_x',
       'total_race_x', 'white_x', 'black_x', 'asian_x', 'not_hispanic_x',
       'total_in_hispanic_x', 'households_below_poverty_x',
       'total_households_x', 'state_x', 'county_x', 'pct_white_x',
       'pct_black_x', 'pct_asian_x', 'hispanic_x', 'pct_hispanic_x',
       'pct_below_poverty_x', 'NAME_y', 'total_race_y', 'white_y', 'black_y',
       'asian_y', 'not_hispanic_y', 'total_in_hispanic_y',
       'households_below_poverty_y', 'total_households_y', 'state_y',
       'county_y', 'pct_white_y', 'pct_black_y', 'pct_asian_y', 'hispanic_y',
       'pct_hispanic_y', 'pct_below_poverty_y', 'NAME', 'total_race', 'white',
       'black', 'asian', 'not_hispanic', 'total_in_hispanic',
       'households_below_poverty', 'total_households', 'state', 'county'])
bronx_2021_merged

,pop_2021,tract,complaints,complaints_norm,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,6661,100,3.0,0.450383,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,4453,200,108.0,24.253312,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,6000,400,46.0,7.666667,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,6038,1600,110.0,18.217953,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,2168,1901,189.0,87.177122,30.627306,40.036900,0.000000,1155,53.274908,28.361582
...,...,...,...,...,...,...,...,...,...,...
342,6351,46207,20.0,3.149110,7.920013,62.068966,4.204062,2038,32.089435,17.654987
343,5879,46208,74.0,12.587175,9.780575,67.970743,0.136078,1913,32.539548,15.912117
344,5039,46209,31.0,6.152014,8.057154,64.397698,0.079381,2214,43.937289,34.946237
345,4213,48401,28.0,6.646095,5.672917,82.150487,0.000000,488,11.583195,6.866614


In [170]:
# remove complaints_norm

bronx_2021_merged = bronx_2021_merged.drop(columns=['complaints_norm'])
bronx_2021_merged

,pop_2021,tract,complaints,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty
0,6661,100,3.0,40.234199,49.121753,2.657259,2166,32.517640,NaN
1,4453,200,108.0,27.734112,28.340445,3.593083,3180,71.412531,19.971264
2,6000,400,46.0,30.150000,24.533333,2.950000,4386,73.100000,4.865848
3,6038,1600,110.0,16.495528,38.456443,1.076515,3364,55.713813,24.051212
4,2168,1901,189.0,30.627306,40.036900,0.000000,1155,53.274908,28.361582
...,...,...,...,...,...,...,...,...,...
342,6351,46207,20.0,7.920013,62.068966,4.204062,2038,32.089435,17.654987
343,5879,46208,74.0,9.780575,67.970743,0.136078,1913,32.539548,15.912117
344,5039,46209,31.0,8.057154,64.397698,0.079381,2214,43.937289,34.946237
345,4213,48401,28.0,5.672917,82.150487,0.000000,488,11.583195,6.866614


In [171]:
#create a column called complaints_norm which is complaints/pop_2021 * 10000

bronx_2021_merged['complaints_norm'] = bronx_2021_merged['complaints'] / bronx_2021_merged['pop_2021'] * 10000
bronx_2021_merged

,pop_2021,tract,complaints,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty,complaints_norm
0,6661,100,3.0,40.234199,49.121753,2.657259,2166,32.517640,NaN,4.503828
1,4453,200,108.0,27.734112,28.340445,3.593083,3180,71.412531,19.971264,242.533124
2,6000,400,46.0,30.150000,24.533333,2.950000,4386,73.100000,4.865848,76.666667
3,6038,1600,110.0,16.495528,38.456443,1.076515,3364,55.713813,24.051212,182.179530
4,2168,1901,189.0,30.627306,40.036900,0.000000,1155,53.274908,28.361582,871.771218
...,...,...,...,...,...,...,...,...,...,...
342,6351,46207,20.0,7.920013,62.068966,4.204062,2038,32.089435,17.654987,31.491104
343,5879,46208,74.0,9.780575,67.970743,0.136078,1913,32.539548,15.912117,125.871747
344,5039,46209,31.0,8.057154,64.397698,0.079381,2214,43.937289,34.946237,61.520143
345,4213,48401,28.0,5.672917,82.150487,0.000000,488,11.583195,6.866614,66.460954


In [172]:
# save as csv

bronx_2021_merged.to_csv('bronx_2021_merged.csv', index=False)

In [178]:
# arrange in ascending order by pct_white

bronx_2021_merged.sort_values(by='pct_white', ascending=True)

,pop_2021,tract,complaints,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty,complaints_norm
66,1608,11701,95.0,0.621891,33.644279,0.000000,1149,71.455224,38.276553,590.796020
298,3264,40400,80.0,1.102941,90.808824,0.000000,312,9.558824,9.175162,245.098039
279,9240,38600,406.0,1.634199,75.822511,0.119048,2137,23.127706,14.622315,439.393939
299,3959,40501,180.0,2.727962,10.027785,3.233140,3540,89.416519,26.690391,454.660268
31,5638,5400,309.0,2.837886,25.540972,2.607308,4016,71.230933,27.419355,548.066690
...,...,...,...,...,...,...,...,...,...,...
221,4480,30900,39.0,77.700893,7.633929,3.102679,710,15.848214,4.427266,87.053571
198,5148,27401,35.0,79.545455,1.845377,2.758353,1623,31.526807,2.771493,67.987568
334,2159,45102,38.0,80.546549,5.048634,2.593793,435,20.148217,5.420354,176.007411
331,1995,44901,19.0,82.005013,8.521303,1.503759,405,20.300752,7.052897,95.238095


In [179]:
# how many NaN values are there the dataframe?

bronx_2021_merged.isnull().sum()

pop_2021             0
tract                0
complaints           4
pct_white            0
pct_black            0
pct_asian            0
hispanic             0
pct_hispanic         0
pct_below_poverty    4
complaints_norm      4
dtype: int64

In [180]:
# look at the rows in the dataframe which have Nan values

bronx_2021_merged[bronx_2021_merged.isnull().any(axis=1)]

,pop_2021,tract,complaints,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty,complaints_norm
0,6661,100,3.0,40.234199,49.121753,2.657259,2166,32.517640,NaN,4.503828
8,4465,2300,NaN,15.655095,35.095185,0.000000,3200,71.668533,44.681960,NaN
62,566,9302,NaN,18.727915,41.519435,3.180212,237,41.872792,NaN,NaN
200,52,27600,NaN,40.384615,59.615385,0.000000,6,11.538462,34.782609,NaN
205,544,28400,12.0,54.227941,27.941176,1.102941,127,23.345588,NaN,220.588235
227,211,31900,3.0,61.611374,28.909953,1.421801,57,27.014218,NaN,142.180095
336,5381,45800,NaN,11.261847,45.400483,0.000000,3259,60.564951,30.211927,NaN


In [181]:
# remove rows with NaN values

bronx_2021_merged = bronx_2021_merged.dropna()
bronx_2021_merged

,pop_2021,tract,complaints,pct_white,pct_black,pct_asian,hispanic,pct_hispanic,pct_below_poverty,complaints_norm
1,4453,200,108.0,27.734112,28.340445,3.593083,3180,71.412531,19.971264,242.533124
2,6000,400,46.0,30.150000,24.533333,2.950000,4386,73.100000,4.865848,76.666667
3,6038,1600,110.0,16.495528,38.456443,1.076515,3364,55.713813,24.051212,182.179530
4,2168,1901,189.0,30.627306,40.036900,0.000000,1155,53.274908,28.361582,871.771218
5,1399,1902,64.0,20.800572,31.165118,3.931380,650,46.461758,19.148936,457.469621
...,...,...,...,...,...,...,...,...,...,...
342,6351,46207,20.0,7.920013,62.068966,4.204062,2038,32.089435,17.654987,31.491104
343,5879,46208,74.0,9.780575,67.970743,0.136078,1913,32.539548,15.912117,125.871747
344,5039,46209,31.0,8.057154,64.397698,0.079381,2214,43.937289,34.946237,61.520143
345,4213,48401,28.0,5.672917,82.150487,0.000000,488,11.583195,6.866614,66.460954


In [182]:
bronx_2021_merged.to_csv('bronx_2021_merged.csv', index=False)